In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import scipy.signal as sig
import pywt

### Set up the Device

In [2]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda:1")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU is available


### Load the Data

In [3]:
dataset_path = './data/'
X_train_valid = np.load(dataset_path + "X_train_valid.npy")
y_train_valid = np.load(dataset_path + "y_train_valid.npy")
person_train_valid = np.load(dataset_path + "person_train_valid.npy")
X_test = np.load(dataset_path + "X_test.npy")
y_test = np.load(dataset_path + "y_test.npy")
person_test = np.load(dataset_path + "person_test.npy")
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


### K-Fold

In [4]:
# some major changes here for the Train_Val_Data function
def Train_Val_Data(X_train_valid, y_train_val):
    '''
    split the train_valid into k folds (we fix k = 5 here)
    return: list of index of train data and val data of k folds
    train_fold[i], val_fold[i] is the index for training and validation in the i-th fold 

    '''
    fold_idx = []
    train_fold = []
    val_fold = []
    train_val_num = X_train_valid.shape[0]
    fold_num = int(train_val_num / 5)
    perm = np.random.permutation(train_val_num)
    for k in range(5):
        fold_idx.append(np.arange(k*fold_num, (k+1)*fold_num, 1))
    for k in range(5):
        val_fold.append(fold_idx[k])
        count = 0
        for i in range(5):
            if i != k:
                if count == 0:
                    train_idx = fold_idx[i]
                else:
                    train_idx = np.concatenate((train_idx, fold_idx[i]))
                count += 1
        train_fold.append(train_idx)

    return train_fold, val_fold

### Data Augmentation Functions

### 1. Window Data

In [5]:
def window_data(X, y, p, window_size, stride):
  '''
  X (a 3-d tensor) of size (#trials, #electrodes, #time series)
  y (#trials,): label 
  p (#trials, 1): person id

  X_new1: The first output stacks the windowed data in a new dimension, resulting 
    in a 4-d tensor of size (#trials x #electrodes x #windows x #window_size).
  X_new2: The second option makes the windows into new trails, resulting in a new
    X tensor of size (#trials*#windows x #electrodes x #window_size). To account 
    for the larger number of trials, we also need to augment the y data.
  y_new: The augmented y vector of size (#trials*#windows) to match X_new2.
  p_new: The augmented p vector of size (#trials*#windows) to match X_new2
 
  '''
  num_sub_trials = int((X.shape[2]-window_size)/stride)
  X_new1 = np.empty([X.shape[0],X.shape[1],num_sub_trials,window_size])
  X_new2 = np.empty([X.shape[0]*num_sub_trials,X.shape[1],window_size])
  y_new = np.empty([X.shape[0]*num_sub_trials])
  p_new = np.empty([X.shape[0]*num_sub_trials])
  for i in range(X.shape[0]):
    for j in range(X.shape[1]):
      for k in range(num_sub_trials):
        X_new1[i,j,k:k+window_size]    = X[i,j,k*stride:k*stride+window_size]
        X_new2[i*num_sub_trials+k,j,:] = X[i,j,k*stride:k*stride+window_size]
        y_new[i*num_sub_trials+k] = y[i]
        p_new[i*num_sub_trials+k] = p[i]
  return X_new1, X_new2, y_new, p_new

### 2. STFT

In [6]:
# Function that computes the short-time fourier transform of the data and returns the spectrogram
def stft_data(X, window, stride):
    '''
    Inputs:
    X - input data, last dimension is one which transform will be taken across.
    window - size of sliding window to take transform across
    stride - stride of sliding window across time-series

    Returns:
    X_STFT - Output data, same shape as input with last dimension replaced with two new dimensions, F x T.
            where F = window//2 + 1 is the frequency axis
            and T = (input_length - window)//stride + 1, similar to the formula for aconvolutional filter.
    t - the corresponding times for the time axis, T
    f - the corresponding frequencies on the frequency axis, F.

    reshape X_STFT (N, C, F, T) to (N, C*F, T) to fit the input of rnn

    Note that a smaller window means only higher frequencies may be found, but give finer time resolution.
    Conversely, a large window gives better frequency resolution, but poor time resolution.

    '''
    noverlap = window-stride
    #print(noverlap)
    if noverlap < 0 :
        print('Stride results in skipped data!')
        return
    f, t, X_STFT = sig.spectrogram(X,nperseg=window,noverlap=noverlap,fs=250, return_onesided=True)
    N, C, F, T = X_STFT.shape
    X_STFT = X_STFT.reshape(N, C*F, T)
    return X_STFT

### 3. CWT

In [7]:
def cwt_data(X, num_levels, top_scale=3):
    '''
    Takes in data, computes CWT using the mexican hat or ricker wavelet using scipy
    Also takes in the top scale parameter.  I use logspace, so scale goes from 1 -> 2^top_scale with num_levels steps.
    Appends to the data a new dimension, of size 'num_levels'
    New dimension corresponds to wavelet content at num_levels different scalings (linear)
    also returns the central frequencies that the scalings correspond to
    input data is N x C X T
    output data is N x C x T x F
    note: CWT is fairly slow to compute

    # EXAMPLE USAGE
    test, freqs = cwt_data(X_train_valid[0:5,:,:],num_levels=75,top_scale=4)
    '''
    scales = np.logspace(start=0,stop=top_scale,num=num_levels)
    out = np.empty((X.shape[0],X.shape[1],X.shape[2],num_levels))
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            coef = sig.cwt(X[i,j,:],sig.ricker,scales)
            out[i,j,:] = coef.T
    freqs = pywt.scale2frequency('mexh',scales)*250
    N, C, T, F = out.shape
    X_CWT = np.transpose(out, (0,1,3,2)).reshape(N, C*F, T)
    return X_CWT

In [8]:
def Aug_Data(X, y, p, aug_type=None, window_size=200, window_stride=20, stft_size=None, stft_stride=None, cwt_level=None, cwt_scale=None):
    if aug_type == None:
        X_aug, y_aug, p_aug = X, y, p
    elif aug_type == "window":
        _, X_aug, y_aug, p_aug = window_data(X, y, p, window_size, window_stride)
    elif aug_type == "stft":
        X_aug = stft_data(X, stft_size, stft_stride)
        y_aug, p_aug = y, p
    elif aug_type == 'cwt':
        X_aug = cwt_data(X, cwt_level, cwt_scale)
        y_aug, p_aug = y, p
    
    return X_aug, y_aug, p_aug

### Customized Dataset

In [9]:
class EEG_Dataset(Dataset):
    '''
    use use fold_idx to instantiate different train val datasets for k-fold cross validation

    '''
    def __init__ (self, X_train=None, y_train=None, p_train=None, X_val=None, y_val=None, p_val=None, X_test=None, y_test=None, p_test=None, mode='train'):
        if mode == 'train':
            self.X = X_train
            self.y = y_train- 769
            self.p = p_train
            
        elif mode == 'val':
            self.X = X_val
            self.y = y_val- 769
            self.p = p_val

        elif mode == 'test':
            self.X = X_test
            self.y = y_test - 769        
            self.p = p_test

    def __len__(self):
        return (self.X.shape[0])
    
    def __getitem__(self, idx):
        '''
        X: (augmented) time sequence 
        y: class label
        p: person id

        '''
        X = torch.from_numpy(self.X[idx,:,:]).float()
        y = torch.tensor(self.y[idx]).long()
        p = torch.tensor(self.p[idx]).long()
        #p = torch.from_numpy(self.p[idx,:]).long()     
        sample = {'X': X, 'y': y, 'p':p}

        return sample

### Define Basic LSTM

In [10]:
class LSTMnet(nn.Module):
    '''
    Create Basic LSTM:
    2 layers

    TODO: make number of layers, dropout, activation function, regularization all params
    see ex: https://blog.floydhub.com/gru-with-pytorch/
    '''

    def __init__(self, input_size, hidden_size, output_dim, dropout):
        super(LSTMnet, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_dim = output_dim
        self.rnn = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=2, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_dim)
    
    def forward(self, x, h=None):
        x = x.permute(2, 0, 1)
        if type(h) == type(None):
            out, hn = self.rnn(x)
        else:
            out, hn = self.rnn(x, h.detach())
        out = self.fc(out[-1, :, :])
        return out

### Define CNN + LSTM

In [18]:
class CNNLSTMnet(nn.Module):
    '''
    CNN + LSTM
    
    '''
    def __init__(self, cnn_input_size, rnn_input_size, hidden_size, output_dim, dropout):
        super(CNNLSTMnet, self).__init__()
        self.cnn_input_size = cnn_input_size
        self.rnn_input_size = rnn_input_size
        self.hidden_size = hidden_size
        self.output_dim = output_dim
        self.cnn = nn.Sequential(
            nn.Conv1d(cnn_input_size, rnn_input_size, kernel_size=5, stride=2),
            nn.BatchNorm1d(rnn_input_size),
            nn.ReLU(),
        )
        self.rnn = nn.LSTM(input_size=rnn_input_size, hidden_size=hidden_size, num_layers=2, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_dim)
    
    def forward(self, x, h=None):
        out = self.cnn(x)
        out = out.permute(2,0,1)
        if type(h) == type(None):
            out, hn = self.rnn(out)
        else:
            out, hn = self.rnn(out, h.detach())
        out = self.fc(out[-1, :, :])
        return out

### Define Basic GRU

In [19]:
class GRUnet(nn.Module):
    '''
    Create Basic GRU:
    2 layers

    TODO: make number of layers, dropout, activation function, regularization all params
    see ex: https://blog.floydhub.com/gru-with-pytorch/
    '''

    def __init__(self, input_size, hidden_size, output_dim, dropout):
        super(GRUnet, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_dim = output_dim
        self.rnn = nn.GRU(input_size=input_size, hidden_size=hidden_size, num_layers=2, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_dim)
    
    def forward(self, x, h=None):
        x = x.permute(2, 0, 1)
        if type(h) == type(None):
            out, hn = self.rnn(x)
        else:
            out, hn = self.rnn(x, h.detach())
        out = self.fc(out[-1, :, :])
        return out

### RNN Initialization

In [21]:
def InitRNN(rnn_type="LSTM", input_size=22, rnn_input_size=40, hidden_size=50, output_dim=4, dropout=0.5, lr=1e-3):
    '''
    Function to initialize RNN
    
    input: RNN type(LSTM, GRU), and other params if neccessary (regularization, acitvation, dropout, num layers, etc.)

    output: model, criterion, optimizer

    TODO: Eventually should also take in params such as dropout, number of layers, and activation function(s), etc.
    '''

    if rnn_type=="LSTM":
        model = LSTMnet(input_size=input_size, hidden_size=hidden_size, output_dim=output_dim, dropout=dropout).to(device)

    elif rnn_type=="GRU":
        model = GRUnet(input_size=input_size, hidden_size=hidden_size, output_dim=output_dim, dropout=dropout).to(device)
    
    elif rnn_type=="CNNLSTM":
        model = CNNLSTMnet(cnn_input_size=input_size, rnn_input_size=rnn_input_size, hidden_size=hidden_size, output_dim=output_dim, dropout=dropout).to(device)


    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    return model, criterion, optimizer


### K-Fold Training and Cross Validation

In [22]:
def TrainRNN(trainloader, valloader, num_epochs=20, verbose=True, aug_type=None):
    val_acc_list = []
    for ep in range(num_epochs):
        tstart = time.time()
        running_loss = 0.0
        correct, total = 0, 0
        for idx, batch in enumerate(EEG_trainloader):
            optimizer.zero_grad()
            X = batch['X'].to(device)
            y = batch['y'].to(device)
            output = model(X)
            loss = criterion(output, y)
            running_loss += loss.item()
            loss.backward()
            optimizer.step()
            pred = torch.argmax(output, dim=1)
            correct += torch.sum(pred == y).item()
            total += y.shape[0]
        train_acc = correct / total
        train_loss = running_loss
        print ('val...')
        '''
        The validation need to be customized according to the data augmenation type
        for stft and cwt: they didn't increase the number of trials, we can directly pass the augmented data to the model
        for window: it increase the number of trials, we need to do a voting for different subsequences in one trial
        
        '''
        if aug_type == 'window':
            correct, total = 0, 0
            for idx, batch in enumerate(EEG_valloader):
                #X = batch['X'].permute(2, 0, 1).to(device)
                X = batch['X'].to(device)
                y = batch['y'].to(device)
                vote_idx = np.random.choice(1000-window_size, vote_num)
                vote_pred = np.zeros(y.shape[0])
                for i in range(len(vote_idx)):
                    X_sub = X[:,:,vote_idx[i]:vote_idx[i]+window_size]
                    output = model(X_sub)
                    pred = torch.argmax(output, dim=1)
                    if i == 0:
                        vote_matrix = np.asarray(pred.cpu().view(-1, 1))
                    else:
                        vote_matrix = np.hstack((vote_matrix, np.asarray(pred.cpu().view(-1,1))))
                for row in range(y.shape[0]):
                    vote_pred[row] = np.bincount(vote_matrix[row, :]).argmax()
                vote_pred = torch.from_numpy(vote_pred).long()
                correct += torch.sum(vote_pred == y.cpu()).item()
                total += y.shape[0]
            val_acc = correct / total        
        else:
            correct, total = 0, 0
            for idx, batch in enumerate(EEG_valloader):
                X = batch['X'].to(device)
                y = batch['y'].to(device)
                output = model(X)                    
                pred = torch.argmax(output, dim=1)
                correct += torch.sum(pred == y.cpu()).item()
                total += y.shape[0]
            val_acc = correct / total
        tend = time.time()
        if verbose:
            print('epoch: {:<3d}    time: {:<3.2f}    loss: {:<3.3f}    train acc: {:<1.3f}    val acc: {:<1.3f}'.format(ep+1, tend - tstart, train_loss, train_acc, val_acc))
        val_acc_list.append(val_acc)
    best_val_acc = max(val_acc_list)
    return best_val_acc

## Pipeline

### 1.Split the data to train and validation

In [23]:
train_fold, val_fold = Train_Val_Data(X_train_valid, y_train_valid)


### 2. Initialize the model

In [24]:
# indicate hyperparameters here
model, criterion, optimizer = InitRNN(rnn_type='CNNLSTM')


### 3. Do K-Fold training and validation

In [25]:
aug_type = "window"
window_size = 200
vote_num = 20
best_val_acc = 0.0
for k in range(5):
    # indicate hyperparameters here
    model, criterion, optimizer = InitRNN(rnn_type='CNNLSTM')
    print ('fold {}'.format(k+1))
    X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
    X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]
    X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type)
    if aug_type != 'window':
        X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
    EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
    EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
    EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
    EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
    best_val_acc += TrainRNN(EEG_trainloader, EEG_valloader, aug_type=aug_type) / 5
print ('average best validation accuracy of 5 folds is :{}'.format(best_val_acc))

fold 1
val...
epoch: 1      time: 17.48    loss: 727.555    train acc: 0.287    val acc: 0.253
val...
epoch: 2      time: 17.27    loss: 719.593    train acc: 0.313    val acc: 0.326
val...
epoch: 3      time: 17.84    loss: 706.714    train acc: 0.344    val acc: 0.407
val...
epoch: 4      time: 17.23    loss: 683.983    train acc: 0.382    val acc: 0.416
val...
epoch: 5      time: 17.26    loss: 660.820    train acc: 0.415    val acc: 0.437
val...
epoch: 6      time: 17.35    loss: 641.622    train acc: 0.436    val acc: 0.390
val...
epoch: 7      time: 17.39    loss: 619.953    train acc: 0.463    val acc: 0.463
val...
epoch: 8      time: 17.46    loss: 598.641    train acc: 0.492    val acc: 0.482
val...
epoch: 9      time: 17.56    loss: 577.860    train acc: 0.515    val acc: 0.397
val...
epoch: 10     time: 17.61    loss: 554.077    train acc: 0.540    val acc: 0.407
val...
epoch: 11     time: 17.42    loss: 533.734    train acc: 0.564    val acc: 0.435
val...
epoch: 12     time

KeyboardInterrupt: 

In [29]:
X_test, y_test, p_test = X_test, y_test, person_test
if aug_type == 'window':
    EEG_testset = EEG_Dataset(X_train, y_train, p_train, X_val, y_val, p_val, X_test, y_test, p_test, mode='test')
    EEG_testloader = DataLoader(EEG_testset, batch_size=128, shuffle=False)
    correct, total = 0, 0
    for idx, batch in enumerate(EEG_testloader):
        X = batch['X'].to(device)
        y = batch['y'].to(device)
        vote_idx = np.random.choice(1000-window_size, vote_num)
        vote_pred = np.zeros(y.shape[0])
        for i in range(len(vote_idx)):
            X_sub = X[:,:,vote_idx[i]:vote_idx[i]+200]
            output = model(X_sub)
            pred = torch.argmax(output, dim=1)
            if i == 0:
                vote_matrix = np.asarray(pred.cpu().view(-1, 1))
            else:
                vote_matrix = np.hstack((vote_matrix, np.asarray(pred.cpu().view(-1,1))))
            for row in range(y.shape[0]):
                vote_pred[row] = np.bincount(vote_matrix[row, :]).argmax()
        vote_pred = torch.from_numpy(vote_pred).long()
        correct += torch.sum(vote_pred == y.cpu()).item()
        total += y.shape[0]
    test_acc = correct / total 
else:
    X_test, y_test, p_test = Aug_Data(X_test, y_test, p_test)
    EEG_testset = EEG_Dataset(X_test=X_test, y_test=y_test, p_test=p_test, mode='test')
    EEG_testloader = DataLoader(EEG_testset, batch_size=128, shuffle=False)    
    correct, total = 0, 0
    for idx, batch in enumerate(EEG_testloader):
        X = batch['X'].to(device)
        y = batch['y'].to(device)
        output = model(X)                    
        pred = torch.argmax(output, dim=1)
        correct += torch.sum(pred == y.cpu()).item()
        total += y.shape[0]
    test_acc = correct / total
print ('Testing Accuracy: {:.4f}'.format(test_acc))


Testing Accuracy: 0.4898
